# Model I/O - Prompt ExampleSelector 少样本提示，示例选择器

## LengthBasedExampleSelector 按长度示例选择器

引入类

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [ ]:
# 创建一个包含词语及其反义词的示例集合。
examples = [
    {"input": "光明", "output": "黑暗"},
    {"input": "快乐", "output": "悲伤"},
    {"input": "上升", "output": "下降"},
    {"input": "胜利", "output": "失败"},
    {"input": "繁荣", "output": "衰败"},
]

# 定义如何将示例数据格式化为字符串的模板。
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="词语: {input}\n反义词: {output}",
)

# 使用LengthBasedExampleSelector选择符合长度要求的示例。
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    # 假设我们希望每个示例的格式化字符串长度不超过10个字符。
    max_length=10,
)

# 创建FewShotPromptTemplate，动态地根据输入选择示例并生成提示。
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="根据词语说出反义词大赛",
    suffix="轮到你了，词语: {input}\n反义词:",
    input_variables=["input"],
)


print(dynamic_prompt.format(input="努力"))

## MaxMarginalRelevanceExampleSelector 基于最大边际相关性的示例选择器

安装向量数据库FAISS

In [ ]:
! pip install faiss-cpu


引入类

In [ ]:

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings

import os

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)

# 创建反义词任务的示例
examples = [
    {"input": "黑暗", "output": "光明"},
    {"input": "寒冷", "output": "温暖"},
    {"input": "干燥", "output": "湿润"},
    {"input": "安静", "output": "喧闹"},
    {"input": "平滑", "output": "粗糙"},
]

# 配置 OpenAI 服务
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  ## 设置openai的key
OPENAI_API_BASE = os.getenv('OPENAI_BASE_URL')  ## 更换为代理地址
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_BASE_URL"))

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 用于生成语义相似性嵌入的嵌入类
    OpenAIEmbeddings(openai_api_base=OPENAI_API_BASE, openai_api_key=OPENAI_API_KEY),
    # 用于存储嵌入并进行相似性搜索的VectorStore类
    FAISS,
    # 要生成的示例数量
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # 我们提供一个示例选择器，而不是示例
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
)

# 如果输入是一个情感或属性，应该首先选择相关的示例
print(mmr_prompt.format(adjective="快乐"))



## SemanticSimilarityExampleSelector 基于相关性的示例选择器

引入类

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings

import os

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)

# 创建反义词任务的示例
examples = [
    {"input": "黑暗", "output": "光明"},
    {"input": "寒冷", "output": "温暖"},
    {"input": "干燥", "output": "湿润"},
    {"input": "安静", "output": "喧闹"},
    {"input": "平滑", "output": "粗糙"},
]

# 配置 OpenAI 服务
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  ## 设置openai的key
OPENAI_API_BASE = os.getenv('OPENAI_BASE_URL')  ## 更换为代理地址
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_BASE_URL"))

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 用于生成语义相似性嵌入的嵌入类
    OpenAIEmbeddings(openai_api_base=OPENAI_API_BASE, openai_api_key=OPENAI_API_KEY),
    # 用于存储嵌入并进行相似性搜索的VectorStore类
    FAISS,
    # 要生成的示例数量
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # 我们提供一个示例选择器，而不是示例
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
)

# 如果输入是一个情感或属性，应该首先选择相关的示例
print(mmr_prompt.format(adjective="快乐"))

